<a href="https://colab.research.google.com/github/besmets/RSE_course/blob/main/RSE_Lecture_04_Digital_Image_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REMOTE SENSING OF THE ENVIRONMENT - LECTURE 4
# Introduction to Digital Image Correlation
(c) Vrije Universiteit Brussel, Prof. Dr. Benoît SMETS - 2023-2024

------------
<br>

### Objectives of this tutorial:
1. Learn the basic principles of digital image correlation.
2. Discover slow-moving landslides

### PREREQUISITES
- Computer with internet connection and web browser.  
- Google GMAIL account.
- Access to Google Earth Engine

### TO START THE EXERCISE

To start using this notebook, first copy it on your drive.
--> On the upper toolbar, click on "Copy to Drive".

<br>
To make this tutorial work, you have to systematically run all cells containing code lines, from the first to the last one (Type SHIFT+ENTER to run a selected cell).  

<br>

------------

## Part 1 – Extracting series of Sentinel-2 images from Google Earth Engine  


In [ ]:
# import modules
import ee
import geemap

# Authenticate
ee.Authenticate()
ee.Initialize(project='ee-bsmets')

In [ ]:
# 1. Define your region and period of interest
# --------------------------------------------

locx = -72.15
locy = -16.37
roi = ee.Geometry.Rectangle([-72.16517, -16.37842, -72.14676, -16.36055])
start = ee.Date("2021-01-01")
finish = ee.Date("2022-01-01")

# 2. Filter the image collection
# -------------------------------

collection = "COPERNICUS/S2_SR_HARMONIZED"
s2_coll = ee.ImageCollection(collection).filterDate(start, finish).filterBounds(roi)

# 3. List the images available in the filtered collection
# --------------------------------------------------------

count = s2_coll.size()                  # Get the number of images available in the filtered collection
img_nbr = int(count.getInfo())          # Transform this value into an integer

# Display the number of images
print('NUMBER OF IMAGES AVAILABLE FOR THE SELECTED DATE RANGE')
print('------------------------------------------------------')
print(str(count.getInfo()) + ' image(s)')
print(' ')

# Display the list of images by date
print('DATETIME AVAILABLE')
print('------------------')
for i in range (0, img_nbr):   # Make a loop listing the date of the images in the filtered collection
    image = ee.Image(s2_coll.toList(s2_coll.size()).get(i))
    ee_date = ee.Date(image.get('system:time_start'))
    print(str(i) + ' = ' + ee_date.format().getInfo())

In [ ]:
# 4. Select two images and display them on the map
# -------------------------------------------------

# Select and load the images
image1 = ee.Image(s2_coll.toList(s2_coll.size()).get(0))
image2 = ee.Image(s2_coll.toList(s2_coll.size()).get(140))

# Define the visualisation parameters
visParams = {'bands': ['B12', 'B11', 'B8A'], 'min': -150, 'max': 4000}

# Create the map and add the selected images
Map = geemap.Map()                             # Load the map
Map.setCenter(locx, locy, 12)                  # Center the map and select the zoom
Map.addLayer(image1.clip(roi), visParams)      # Add Image 1 to the map
Map.addLayer(image2.clip(roi), visParams)      # Add Image 2 to the map

Map

Let's tune a little bit the display here by adding more options:

In [ ]:
# Create various rgb-compositions
visible = ['B4', 'B3', 'B2']
vnir = ['B8', 'B4', 'B3']
swir = ['B12', 'B11', 'B8A']

# Select the basemap in the background ('ROADMAP', 'SATELLITE', 'TERRAIN', 'HYBRID', 'OpenStreetMap')
Map = geemap.Map(basemap='TERRAIN', center=[locy, locx], zoom=14)   # !Note locx and locy are inverted here!

visParams = {'bands': vnir, 'min': -150, 'max': 4000}

Map.addLayer(image1.clip(roi), visParams, 'S2_VNIR_2021-01-04')      # Add Image 1 to the map, with its name specified
Map.addLayer(image2.clip(roi), visParams, 'S2_VNIR_2021-12-30')     # Add Image 2 to the map, with its name specified

Map

### Sentinel-2 image export

In [ ]:
# Select the spectral bands for each image and crop to only cover the ROI
band_selection = ['B8', 'B4', 'B3']
S2_image_1 = image1.select(band_selection).clip(roi)
S2_image_2 = image2.select(band_selection).clip(roi)

# Export arguments
export_folder = '__RSE_colab_data'    # just direct folders (no subfolders)
fileNamePrefix1 = 'S2_VNIR_20210104'  # Name for export image 1
fileNamePrefix2 = 'S2_VNIR_20211230'  # Name for export image 2
description1 = 'Selected_S2_image_1'  # Export task description for image 1
description2 = 'Selected_S2_image_2'  # Export task description for image 2
pixel_size = 10;                      # in metres
chosen_crs = 'EPSG:32718'             # Projected Coordinate System (WGS84 UTM 18 South)
file_Format = 'GeoTIFF'

# Export the two images
geemap.ee_export_image_to_drive(S2_image_1, description=description1, folder=export_folder, fileNamePrefix=fileNamePrefix1, scale=pixel_size, crs=chosen_crs, fileFormat=file_Format)
geemap.ee_export_image_to_drive(S2_image_2, description=description2, folder=export_folder, fileNamePrefix=fileNamePrefix2, scale=pixel_size, crs=chosen_crs, fileFormat=file_Format)

--------------

## PART 2 — Digital Image Correlation (DIC) with AROSICS

In this part, we will use the downloaded images and AROSICS, a tool initially designed for image co-registration, to perform DIC.  

#### 2.1. (Down)loading the packages and the images

In [ ]:
# Install the missing packages

!pip install arosics
!pip install rasterio

In [ ]:
# Import packages

from arosics import COREG_LOCAL
import pandas as pd
import datetime
from google.colab import drive

# Mount Google Drive and specify the image paths

drive.mount('/content/gdrive')

ref = 'gdrive/MyDrive/__RSE_colab_data/Lecture_04/S2_VNIR_20210104.tif'
target = 'gdrive/MyDrive/__RSE_colab_data/Lecture_04/S2_VNIR_20211230.tif'

#### 2.2. Setup the image correlation

In [ ]:
# =============================================================================
# =============================   SETUP SECTION   =============================
# =============================================================================

im_reference= ref
im_target = target

grid_resolution = 3   # resolution of the grid of spatial shifts
win_size = (20, 20)  # size of the moving window
max_shift=100 # Max allowed shift between target and ref images (in m)
out_format= 'GTiff' # Output format

# =============================================================================

#### 2.3. Perform the image correlation and export results in a CSV file

In [ ]:
# =============================================================================
# ============================   AROSICS SECTION   ============================
# =============================================================================

### CALCULATE AND APPLY LOCAL CO-REGISTRATION ###

  # Print basic info

print('============================================================')
print('===        Digital Image Correlation with AROSICS        ===')
print('============================================================')
print(' ')
print('Reference image: ' + str(ref))
print('Target image: ' + str(target))
print(' ')

  # Set the parameters of the local coregistration
kwargs = {
    'grid_res' : grid_resolution,
    'window_size' : win_size,
    'path_out' : im_target[:-4] + '_local_coreg.tif',
    'fmt_out' : out_format,
    'max_shift' : max_shift,
}

  # Create the local co-registration variable
print (' ')
print(' ### CALCULATE SHIFT ### ')
print('--> Start time: ' + str(datetime.datetime.now()))
print(' ')
CRL = COREG_LOCAL(im_reference, im_target,**kwargs)
print(' ')
print('--> Stop time: ' + str(datetime.datetime.now()))
print('#########################')


  # Apply local shifts to target image
# CRL.correct_shifts()

### SAVING SHIFT TABLE IN CSV FORMAT ###

  # CSV
print(' ')
print(' ### SAVING SHIFT POINTS IN A CSV FILE ### ')
print('--> Start time: ' + str(datetime.datetime.now()))
table_out = im_target[:-4] + '_CRL_shift_table.csv'
pd.DataFrame(CRL.CoRegPoints_table).to_csv(table_out, sep='\t')
print(' ')
print('--> Shift points saved in CSV file')
print('--> Stop time: ' + str(datetime.datetime.now()))

print(' ')
print('=====================================')
print(' PROCESSING STEPS ENDED SUCCESSFULLY ')
print('=====================================')

#### 2.4. Plot the results

Please, adapt the setup section accordingly!

In [ ]:
import rasterio
from rasterio.plot import show
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.close('all')

# SETUP SECTION
# --------------

# Provide the working directory with '/' at the end
work_path = 'gdrive/MyDrive/__RSE_colab_data/Lecture_04/'

# Provide the name of the csv file
csv_name = 'S2_VNIR_20211230_CRL_shift_table.csv'

# Provide the name of the background image
bg_image = 'S2_VNIR_20210104.tif'

# Choose point size in the figure (suggested: 2 to 5)
point_size = 2

# Stretch or not the colorscale of the points
stretch = 'yes'   # Select 'yes' or 'no'
cmin = 0
cmax = 20

# Projected coordinate system
selected_crs = 'EPSG:32718'

# Display or save the figure
display = 'show'   # Select 'show' or 'save'

# If 'save' option is selected, choose the figure name with extension
figname = 'shift_plot_SCATTER.png'


# PLOTTING SECTION
# -----------------

# Load the background image

raster = rasterio.open(f'{work_path}{bg_image}', crs=selected_crs)

# Load the CSV file, remove NaN values, and create a geotable

raw_table = pd.read_csv(f'{work_path}{csv_name}', sep='\t')

NaN_value = -9999.0
vector_table = raw_table.loc[raw_table['X_SHIFT_M']>NaN_value].copy()

geotable = gpd.GeoDataFrame(vector_table, geometry=gpd.points_from_xy(vector_table.X_MAP, vector_table.Y_MAP))
geotable.crs = selected_crs

# Vector variables
X = geotable['X_MAP']      # X coordinate
Y = geotable['Y_MAP']      # Y coordinate
C = geotable['ABS_SHIFT']  # Absolute shift value (for the colormapping)

# Colormaps
vector_cmap = 'viridis'
raster_cmap = 'Greys_r'   # The '_r' reverse the colormap

# Plotting

fig, ax = plt.subplots()
show((raster, 1), cmap=raster_cmap, ax=ax, alpha=0.5)
if stretch in ['yes', 'Yes', 'y', 'Y']:
    q = ax.scatter(X, Y, s=point_size, c=C, cmap=vector_cmap, vmin=cmin, vmax=cmax)
else:
    q = ax.scatter(X, Y, s=point_size, c=C, cmap=vector_cmap)

plt.title('Absolute shift (m)', fontweight='bold')
plt.xlabel('Easting (m)', fontweight='bold')
plt.ylabel('Northing (m)', fontweight='bold')
plt.colorbar(q)     # Add the colorbar to the right
plt.axis('equal')   # Ensure that X and Y scales are equals
plt.tight_layout()  # Ensure that everything is within the figure

if display == 'show':
    plt.show()
elif display == 'save':
    plt.savefig(f'{work_path}{figname}', dpi=600)